In [269]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [290]:
import numpy as np

class RidgeRegressionGD:

    def __init__(self, alpha: np.float64 = 1.0, iterations: np.float64 = 100, learning_rate: np.float64 = 0.01):
        self.alpha_                  = alpha
        self.coef_: np.array         = None
        self.intercept_: np.float64  = None 
        self.iterations_             = iterations
        self.learning_rate_          = learning_rate

    def fit(self, X_train, y_train):

        BETA =  np.zeros(shape=(X_train.shape[1] + 1, 1))
        
        X_train = np.insert(X_train, obj=0, values=1, axis=1)
        y_train = np.reshape(y_train, shape=(X_train.shape[0], 1))

        for i in range(self.iterations_):
            der_BETA = ( np.dot( X_train.T, (np.dot(X_train, BETA) - y_train) )  + (self.alpha_ * BETA) ) / X_train.shape[0]
            BETA -= self.learning_rate_ * der_BETA
        
        self.intercept_ = BETA[0, 0]
        self.coef_      = BETA[1:]

        return self

    def predict(self, X_test):

        return np.dot(X_test, self.coef_) + self.intercept_

    def score(self, X_test, y_test, method: str = None):
        y_pred = self.predict(X_test).ravel()

        assert(y_pred.shape == y_test.shape)
        
        ss_total = np.sum((y_test - np.mean(y_test)) ** 2)
        ss_residual = np.sum((y_test - y_pred) ** 2)

        R2 = 1 - (ss_residual / ss_total)
        n, k = X_test.shape

        if method == "adj":
            return 1 - ( (1 - R2) * (n - 1) / (n - 1 - k) )

        return 1 - (ss_residual / ss_total)

In [315]:
ALPHA = 3
LEARNING_RATE = 0.05
ITERATIONS = 5000

In [316]:
X, y = load_diabetes(return_X_y=True)

In [317]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y, 
    train_size=0.8,
    random_state=42
)

In [318]:
lr_me = RidgeRegressionGD(alpha=ALPHA, iterations=ITERATIONS, learning_rate=LEARNING_RATE)
lr_sk = Ridge(alpha=ALPHA, max_iter=ITERATIONS, solver='sparse_cg')

In [319]:
lr_me.fit(X_train, y_train)
lr_sk.fit(X_train, y_train)

Ridge(alpha=3, max_iter=5000, solver='sparse_cg')

In [320]:
lr_me.score(X_test, y_test), lr_sk.score(X_test, y_test)

(np.float64(0.31204815957055265), 0.3158542845449235)